# Woodler Formulation

Statement of the use case: https://www.mipwise.com/use-cases/woodler


## Formulation


### Input Data Model

#### Indices
- $I$: Collection of sites.
- $K$: Collection of trucks.
- $L$: Collection of orders.

#### Parameters
* $u_k$: Capacity of truck $k$.
* $cf_k$: Fixed cost of truck $k$.
* $cv_k$: Variable cost of truck $k$.
* $v_{l}$: Volume of order $l$.
* $s_{l}$: Site of order $l$.
* $td_{ij}$: Transit distance from site $i$ to $j$.
* $tt_{ij}$: Transit time from site $i$ to $j$.

### Decision Variables
* $x_{ijk}$: $1$ if truck $k$ goes from site $i$ to site $j$; $0$ otherwise.
* $y_{kl}$: $1$ if truck $k$ delivers order $l$; $0$ otherwise.
* $z_k$: $1$ if truck $k$ is used; $0$ otherwise.

### Constraints

* C1) Every truck departs from the DC:
    $$ \sum_{j} x_{i_0jk} = z_k, \quad i_0=DC, \;\forall k $$
    
    If truck $k$ is used ($z_k=1$), then it must depart from the DC, otherwise ($z_k=0$) it must not.


* C2) At most one origin:
$$\sum_i x_{ijk} \leq 1, \quad \forall k, j.$$

* C3) At most one destination:
$$\sum_j x_{ijk} \leq 1, \quad \forall k, i.$$

* C4) Flow balance:
$$\sum_i x_{ihk} = \sum_j x_{hjk}, \quad \forall k, h\in I.$$

* C5) Must deliver every order:
    $$ \sum_{k} y_{kl} = 1, \quad \forall l $$
    That is, given a order $l$, there must be exactly one truck $k$ assigned.


* C6) Relationship between $y$ and $x$:
    $$ y_{kl} \leq \sum_{i} x_{is_lk}, \quad \forall k,l $$
    
    That is, if truck $k$ is assigned to order $l$ ($y_{kl} = 1$), then it must visit the site of order $l$, which is $s_l$. **Equality sign cannot be used here**, since it could lead to infeasibility, or at least to bad behavior. The reason is that, even if truck $k$ isn't assigned to a order $l$, in which case $y_{kl}=0$, it still can visit site $s_l$ due to another order (with the same site) assigned to it, or even because the truck may need to simply pass by that site to get anywhere else.
    

* C7) Relationship between $y$ and $z$:
    $$ y_{kl} \leq z_{k}, \quad \forall k,l $$
    
    In words: if truck $k$ is assigned to some order $l$ ($y_{kl}=1$), then we must use this truck ($z_k=1$). Same idea in the opposite direction: if truck $k$ is not used ($z_k=0$), then it cannot be assigned to any order ($y_{kl}=0, \forall l$).
    
    
* C8) Truck capacity:
    $$ \sum_{l} v_l \cdot y_{kl} \leq u_k, \quad \forall k$$


### Additional Constraints

Some additional constraints that may be redundant with the previous ones, but might help the solver.

* C9) Relationship between $x$ and $z$:
    $$ x_{ijk} \leq z_k, \quad \forall i,j,k $$
    
    If truck $k$ is not used ($z_k=0$), then it cannot move ($x_{ijk}=0, \forall i,j$). This prevents the solver from prescribing some vehicle to move (turn on some $x_{ijk}$) without using that vehicle, which wouldn't happen anyway due to our objective function, which tries to turn off as many $x_{ijk}$ and $z_k$ variables as possible.

### Sub-tour elimination

If necessary, we add sub-tour elimination constraints, either DFJ or MTZ:

* DFJ sub-tour elimination:
    $$ \sum_{i,j \in S} x_{ijk} \leq |S| - 1, \quad \forall k, \;\forall S \subseteq I\backslash \{DC\}$$ 
    
    Notice that it's important to restrict the subsets of nodes (over which we eliminate sub-tours) to be only those sets not containing the DC. The reason is that, since we don't know which nodes each vehicle $k$ will visit, it's perfectly possible to have a cycle like DC -> S1 -> S2 -> DC, not including site S3, for example, in case the vehicle doesn't have to deliver at S3. But we couldn't have this previous cycle AND also, say, S4 <-> S5, and this bad behavior is indeed impossible by the constraint above (for $S=\{S4, S5\}$).
    

* MTZ sub-tour elimination:
    $$ u_{ik} + M\cdot x_{ijk} + 1 \leq u_{jk}, \quad \forall i,j,k, \; j\neq DC $$
    $$ u_{i_0, k} = z_k, \quad \forall k, \; i_0 = DC $$
    where $u_{ik}$ is an additional (integer, non-negative) variable.


### Objective
The objective is to minimize the total cost.
* $\text{fixed_cost} = \sum_{k} cf_k \cdot z_k$
* $\text{variable_cost} = \sum_{ijk} cv_k\cdot td_{ij}\cdot x_{ijk}$
$$\min{\text{fixed_cost} + \text{variable_cost}}.$$
